In [3]:
import numpy as np
import pathlib as pth
import xarray as xr
import itertools
import importlib
import sys
sys.path.extend(['/home/jleland/coding/projects/autospice', '/home/jleland/coding/projects/flopter'])
from codes import spice as sp

In [4]:
importlib.reload(sp)

<module 'codes.spice' from '/home/jleland/coding/projects/autospice/codes/spice.py'>

In [5]:
spice = sp.Spice()

In [6]:
input_file = pth.Path('/home/jleland/spice/spice2/bin/inputs/sheath_exp/piecewise_hg/angled_recessed.piecewise.inp')
# input_file = pth.Path('/home/jleland/spice/spice2/bin/inputs/sheath_exp/2dscan_angled_recessed.inp')
# input_file = pth.Path('/home/jleland/spice/spice2/bin/inputs/sheath_exp/angled_recessed.inp')
# input_file = pth.Path('/home/jleland/spice/spice2/bin/inputs/baseangledprobe.inp')

In [7]:
spice.is_parameter_scan(input_file)

True

In [10]:
scan_params, parser = spice.get_scanning_parameters(input_file)
scan_params

[{'section': 'plasma',
  'parameter': 'alpha_yz',
  'values': ['-3.0',
   '-4.0',
   '-5.0',
   '-6.0',
   '-8.0',
   '-10.0',
   '-12.0',
   '-15.0',
   '-20.0',
   '-30.0'],
  'length': 10},
 {'section': 'geom',
  'parameter': 'ly',
  'values': ['1351',
   '1103',
   '955',
   '855',
   '731',
   '655',
   '605',
   '584',
   '564',
   '586'],
  'length': 10},
 {'section': 'rectangle1',
  'parameter': 'yhigh',
  'values': ['1351',
   '1103',
   '955',
   '855',
   '731',
   '655',
   '605',
   '584',
   '564',
   '586'],
  'length': 10},
 {'section': 'rectangle2',
  'parameter': 'pot',
  'values': ['-25.0',
   '-18.0',
   '-12.0',
   '-10.0',
   '-9.0',
   '-8.0',
   '-7.0',
   '-6.0',
   '-5.0',
   '-4.0',
   '-3.0',
   '-2.0',
   '-1.0',
   '0.0'],
  'length': 14},
 {'section': 'triangle0',
  'parameter': 'pot',
  'values': ['-25.0',
   '-18.0',
   '-12.0',
   '-10.0',
   '-9.0',
   '-8.0',
   '-7.0',
   '-6.0',
   '-5.0',
   '-4.0',
   '-3.0',
   '-2.0',
   '-1.0',
   '0.0'],
  'l

In [12]:
lengths = set([sp['length'] for sp in scan_params])
lengths

{10, 14}

In [17]:
param_scan_d = 2

print(len(lengths))
print(len(lengths) != param_scan_d) 

print(len(scan_params) != param_scan_d)

2
False
True


In [63]:
all_sp_vals = [sp['values'] for sp in scan_params]
all_sp_lens = [sp['length'] for sp in scan_params]
length_dim_indices = {length: j for j, length in enumerate(lengths)}

print(all_sp_lens)
print(length_dim_indices)

[10, 10, 10, 14, 14, 10, 10]
{10: 0, 14: 1}


In [66]:
all_sp_len_dims = [length_dim_indices[l] for l in all_sp_lens]
all_sp_len_dims

[0, 0, 0, 1, 1, 0, 0]

In [87]:
length_dim_index_map = [[] for length in lengths]
for i, length in enumerate(all_sp_lens):
    length_dim_index_map[length_dim_indices[length]].append(i)
length_dim_index_map

[[0, 1, 2, 5, 6], [3, 4]]

In [59]:
length_dim_inner_prod = list(itertools.product(*[range(length) for length in lengths]))
# arranged_sp_vals = []
arranged_sp_vals = np.stack([
    np.concatenate([
        [all_sp_vals[j][arrangement[dim_i]] 
             for j in length_dim_index_map[dim_i]] 
        for dim_i in range(param_scan_d)]) 
    for arrangement in length_dim_inner_prod
])

In [72]:
arranged_sp_vals = [tuple(all_sp_vals[i][arrangement[d]] for i, d in enumerate(all_sp_len_dims)) for arrangement in length_dim_inner_prod]
        

[('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-18.0', '-18.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-12.0', '-12.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-10.0', '-10.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-9.0', '-9.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-8.0', '-8.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-7.0', '-7.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-6.0', '-6.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-5.0', '-5.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-4.0', '-4.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-3.0', '-3.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-2.0', '-2.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-1.0', '-1.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '0.0', '0.0', '1351', '1351'),
 ('-4.0', '1103', '1103', '-25.0', '-25.0', '1103', '1103'),
 ('-4.0', '1103', '1103', '-18.0', '-18.0', '1103', '1103'),
 ('-4.0', '1103', '1103', '-12.0', '-12.0', '1

In [60]:
list(arranged_sp_vals)

[array(['-3.0', '1351', '1351', '1351', '1351', '-25.0', '-25.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-18.0', '-18.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-12.0', '-12.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-10.0', '-10.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-9.0', '-9.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-8.0', '-8.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-7.0', '-7.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-6.0', '-6.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-5.0', '-5.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-4.0', '-4.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351', '-3.0', '-3.0'],
       dtype='<U5'),
 array(['-3.0', '1351', '1351', '1351', '1351'

## autospice development time!

In [86]:
param_scan_fl = spice.is_parameter_scan(input_file)
param_scan_d = 2

if param_scan_fl:
    # TODO: The use of an input parser is SPICE specific
    raw_scan_params, inp_parser = spice.get_scanning_parameters(input_file)
    lengths = set([sp['length'] for sp in raw_scan_params])
    if len(lengths) != param_scan_d and len(scan_params) != param_scan_d:
        raise ValueError(
            f'Cannot perform {param_scan_d}d parameter scan; "param_scan_dims" should be set to the number of '
            f'different parameter scans ({len(scan_params)}) or the number of different scan lengths '
            f'({len(lengths)}). \n'
            f'Mixed multi-dimensional scanning is not yet supported.')
    
    if len(raw_scan_params) == 1:
        arranged_sp_vals = [tuple([value], ) for value in raw_scan_params[0]['values']]
    else:
        all_sp_vals = [sp['values'] for sp in raw_scan_params]
        if param_scan_d == 1:
            arranged_sp_vals = zip(*all_sp_vals)
        elif len(lengths) == param_scan_d:
            length_dim_indices = {length: j for j, length in enumerate(lengths)}
            all_sp_lens = [sp['length'] for sp in scan_params]
            all_sp_len_dims = [length_dim_indices[l] for l in all_sp_lens]
            
            dim_permutations = list(itertools.product(*[range(length) for length in lengths]))
            arranged_sp_vals = [tuple(all_sp_vals[i][perm[d]] for i, d in enumerate(all_sp_len_dims)) for perm in dim_permutations]
        else:
            arranged_sp_vals = itertools.product(*all_sp_vals)
        
    arranged_sp_vals = list(arranged_sp_vals)
    names = "\', \'".join([f"{sp['section']}/{sp['parameter']}" for sp in raw_scan_params])

    print(f"Submitting a {param_scan_d}d parameter scan! \n"
          f"Scanning over parameter(s) \'{names}\' \n"
          f"with the following values (N={len(arranged_sp_vals)}): \n")
    print_slc = slice(None, None)
    if len(arranged_sp_vals) > 100:
        print_slc = slice(0, 20)
    for i, value in enumerate(arranged_sp_vals[print_slc]):
        print(f'{i+1}) \t{", ".join([v for v in value])}')
#         print(f'{i+1}) \t{value}')
    if len(arranged_sp_vals) > 100:
        print('\n\t· \n\t· \n\t· \n')
        for i, value in enumerate(arranged_sp_vals[-5:]):
            N = len(arranged_sp_vals) - 4
            print(f'{N+i}) \t{", ".join([v for v in value])}')
#         print(f'{i+1}) \t{value}')
    print('\n')
else:
    scan_param = {'values': [None]}
    inp_parser = None

Submitting a 2d parameter scan! 
Scanning over parameter(s) 'plasma/alpha_yz', 'geom/ly', 'rectangle1/yhigh', 'rectangle2/pot', 'triangle0/pot', 'diag_reg3/y_high', 'diag_reg4/y_high' 
with the following values (N=140): 

1) 	-3.0, 1351, 1351, -25.0, -25.0, 1351, 1351
2) 	-3.0, 1351, 1351, -18.0, -18.0, 1351, 1351
3) 	-3.0, 1351, 1351, -12.0, -12.0, 1351, 1351
4) 	-3.0, 1351, 1351, -10.0, -10.0, 1351, 1351
5) 	-3.0, 1351, 1351, -9.0, -9.0, 1351, 1351
6) 	-3.0, 1351, 1351, -8.0, -8.0, 1351, 1351
7) 	-3.0, 1351, 1351, -7.0, -7.0, 1351, 1351
8) 	-3.0, 1351, 1351, -6.0, -6.0, 1351, 1351
9) 	-3.0, 1351, 1351, -5.0, -5.0, 1351, 1351
10) 	-3.0, 1351, 1351, -4.0, -4.0, 1351, 1351
11) 	-3.0, 1351, 1351, -3.0, -3.0, 1351, 1351
12) 	-3.0, 1351, 1351, -2.0, -2.0, 1351, 1351
13) 	-3.0, 1351, 1351, -1.0, -1.0, 1351, 1351
14) 	-3.0, 1351, 1351, 0.0, 0.0, 1351, 1351
15) 	-4.0, 1103, 1103, -25.0, -25.0, 1103, 1103
16) 	-4.0, 1103, 1103, -18.0, -18.0, 1103, 1103
17) 	-4.0, 1103, 1103, -12.0, -12.0, 1103

In [62]:
arranged_sp_vals

[('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '1351'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '1103'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '955'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '855'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '731'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '655'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '605'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '584'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '564'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1351', '586'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1103', '1351'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1103', '1103'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1103', '955'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1103', '855'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1103', '731'),
 ('-3.0', '1351', '1351', '-25.0', '-25.0', '1103', '655'),
 ('-3.0', '1351', '1351', '-25.0', '

In [18]:
stacked_sp_vals = np.array([sp['values'] for sp in scan_param])
stacked_sp_vals

NameError: name 'scan_param' is not defined

In [34]:
ds = xr.concat(
    [xr.Dataset(scan_param[i]).expand_dims(dim=['scan_param']).reset_index('values') 
     for i in range(len(scan_param))], 
    'scan_param')
ds['values_'].values

array([['-2.0', '-3.0', '-4.0', '-5.0', '-6.0', '-8.0', '-10.0', '-12.0',
        '-20.0', '-30.0'],
       ['1849', '1351', '1103', '955', '855', '731', '655', '605', '564',
        '586'],
       ['1849', '1351', '1103', '955', '855', '731', '655', '605', '564',
        '586'],
       ['1849', '1351', '1103', '955', '855', '731', '655', '605', '564',
        '586']], dtype=object)

In [42]:
# for a in itertools.product(*ds['values_'].values):
#     print(a)
list(itertools.product(*stacked_sp_vals))

[('-2.0', '1849', '1849', '1849'),
 ('-2.0', '1849', '1849', '1351'),
 ('-2.0', '1849', '1849', '1103'),
 ('-2.0', '1849', '1849', '955'),
 ('-2.0', '1849', '1849', '855'),
 ('-2.0', '1849', '1849', '731'),
 ('-2.0', '1849', '1849', '655'),
 ('-2.0', '1849', '1849', '605'),
 ('-2.0', '1849', '1849', '564'),
 ('-2.0', '1849', '1849', '586'),
 ('-2.0', '1849', '1351', '1849'),
 ('-2.0', '1849', '1351', '1351'),
 ('-2.0', '1849', '1351', '1103'),
 ('-2.0', '1849', '1351', '955'),
 ('-2.0', '1849', '1351', '855'),
 ('-2.0', '1849', '1351', '731'),
 ('-2.0', '1849', '1351', '655'),
 ('-2.0', '1849', '1351', '605'),
 ('-2.0', '1849', '1351', '564'),
 ('-2.0', '1849', '1351', '586'),
 ('-2.0', '1849', '1103', '1849'),
 ('-2.0', '1849', '1103', '1351'),
 ('-2.0', '1849', '1103', '1103'),
 ('-2.0', '1849', '1103', '955'),
 ('-2.0', '1849', '1103', '855'),
 ('-2.0', '1849', '1103', '731'),
 ('-2.0', '1849', '1103', '655'),
 ('-2.0', '1849', '1103', '605'),
 ('-2.0', '1849', '1103', '564'),
 ('-2

In [44]:
list(zip(*ds['values_'].values))


[('-2.0', '1849', '1849', '1849'),
 ('-3.0', '1351', '1351', '1351'),
 ('-4.0', '1103', '1103', '1103'),
 ('-5.0', '955', '955', '955'),
 ('-6.0', '855', '855', '855'),
 ('-8.0', '731', '731', '731'),
 ('-10.0', '655', '655', '655'),
 ('-12.0', '605', '605', '605'),
 ('-20.0', '564', '564', '564'),
 ('-30.0', '586', '586', '586')]

In [113]:
for j, pv in enumerate(arranged_sp_vals):
    print(j, pv[0])

0 -1.0
1 -2.0
2 -3.0
3 -5.0
4 -10.0
5 -30.0


## Checking the output now

In [14]:
m1d_dir = pth.Path('/home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi')
m1d_run_dirs = list(m1d_dir.glob('alpha*'))

In [15]:
for i, mrd in enumerate(m1d_run_dirs):
    print(i, mrd / 'input.inp')

0 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-2.0/input.inp
1 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-3.0/input.inp
2 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-4.0/input.inp
3 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-5.0/input.inp
4 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-6.0/input.inp
5 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-8.0/input.inp
6 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-10.0/input.inp
7 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-12.0/input.inp
8 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-20.0/input.inp
9 /home/jleland/spice/spice2/bin/data_offline_1/scan_test/1d_scan_multi/alpha_yz_-30.0/input.inp


In [16]:
%matplotlib tk
import flopter.spice.utils as sput
import flopter.spice.inputparser as inp
import matplotlib.pyplot as plt

In [17]:
fig, ax = plt.subplots(10, sharex=True)

for i, mrd in enumerate(m1d_run_dirs):
#     ii = i // 3
#     jj = i % 3
    print(i)
    sput.plot_2d_sim_window(mrd/'input.inp', ax=ax[i])
    parser = inp.InputParser(input_filename=str(mrd/'input.inp'))
    angle = parser.getfloat('plasma', 'alpha_yz')
    ax[i].quiver(427, 131, 1, np.tan(np.radians(angle)), scale=i*2)

0
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
1
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
2
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
3
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
4
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
5
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
6
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
7
{'circle': [], 'triangle': [<Section: t

/home/jleland/coding/environments/python-3.7/lib/python3.7/site-packages/matplotlib/quiver.py:686: RuntimeWarning: divide by zero encountered in double_scalars
  length = a * (widthu_per_lenu / (self.scale * self.width))
